In [45]:
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

conn = pg.connect(host='pgsql-196447.vipserv.org', port='5432', dbname='wbauer_adb', user='wbauer_adb', password='adb2020')




In [46]:
df = pd.read_sql('SELECT * FROM city', con=conn)
print(df)



     city_id                city  country_id         last_update
0          1  A Corua (La Corua)          87 2006-02-15 09:45:25
1          2                Abha          82 2006-02-15 09:45:25
2          3           Abu Dhabi         101 2006-02-15 09:45:25
3          4                Acua          60 2006-02-15 09:45:25
4          5               Adana          97 2006-02-15 09:45:25
..       ...                 ...         ...                 ...
595      596               Zaria          69 2006-02-15 09:45:25
596      597        Zeleznogorsk          80 2006-02-15 09:45:25
597      598         Zhezqazghan          51 2006-02-15 09:45:25
598      599            Zhoushan          23 2006-02-15 09:45:25
599      600          Ziguinchor          83 2006-02-15 09:45:25

[600 rows x 4 columns]


In [47]:
s = "SELECT city, country_id FROM city WHERE country_id = %(polaaand)s"
df = pd.read_sql(s, con=conn, params={'polaaand': 76})
print(df)

              city  country_id
0        Bydgoszcz          76
1      Czestochowa          76
2  Jastrzebie-Zdrj          76
3           Kalisz          76
4           Lublin          76
5            Plock          76
6            Tychy          76
7          Wroclaw          76


ZADANIE 1

In [48]:
s = "SELECT COUNT(category_id) FROM category"
df = pd.read_sql(s, conn)
print(df)

   count
0     16


ZADANIE 2

In [49]:
s = "SELECT name FROM category ORDER BY name ASC"
df = pd.read_sql(s, conn)
print(df)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


ZADANIE 3

In [50]:
s1 = "SELECT release_year FROM film"
df1 = pd.read_sql(s1, conn)
df1_len = df1.shape[0]
print(df1_len)
# najstarszy
s = "SELECT release_year, title FROM film ORDER BY release_year ASC OFFSET %(df1_len)s"
df = pd.read_sql(s, conn, params={'df1_len': df1_len-1})
print(df)

# najnowszy
s = "SELECT release_year, title FROM film ORDER BY release_year ASC LIMIT 1"
df = pd.read_sql(s, conn)
print(df)

1000
   release_year      title
0          2006  Zorro Ark
   release_year            title
0          2006  Chamber Italian


ZADANIE 4

In [51]:
import datetime


s = """SELECT COUNT(rental_date) FROM rental WHERE rental_date 
BETWEEN %(rental_date)s AND %(rd2)s"""
df = pd.read_sql(s, conn, params={'rental_date': datetime.datetime(2005, 7, 1, 0, 0, 1), 'rd2' : datetime.datetime(2005, 8, 1, 23, 59, 59)})
print(df)

   count
0   7380


Zadanie 5

In [52]:
s = "SELECT COUNT(rental_date) FROM rental WHERE rental_date BETWEEN %(rental_date)s AND %(rd2)s"
df = pd.read_sql(s, conn, params={'rental_date': datetime.datetime(2010, 1, 1, 0, 0, 0), 'rd2' : datetime.datetime(2011, 2, 1, 23, 59, 59)})
print(df)

   count
0      0


Zadanie 6

In [53]:
s = "SELECT MAX(amount) FROM payment"
df = pd.read_sql(s, conn)
print(df)

     max
0  11.99


Zadanie 7

In [54]:
s = """
SELECT first_name, last_name FROM customer WHERE address_id IN
(SELECT address_id FROM address WHERE city_id IN
(SELECT city_id FROM city WHERE country_id IN
(SELECT country_id FROM country WHERE country IN (%(Poland)s, %(Nigeria)s, %(Bangladesh)s))))
"""
df = pd.read_sql(s, conn, params={'Poland':'Poland', 'Nigeria': 'Nigeria', 'Bangladesh': 'Bangladesh'})
print(df)

   first_name  last_name
0     Johnnie   Chisholm
1     Wallace      Slone
2      Bertha   Ferguson
3     Russell    Brinson
4    Michelle      Clark
5       Brian      Wyman
6      Tracey    Barrett
7     Stephen     Qualls
8       Ruben      Geary
9   Constance       Reid
10      Carol     Garcia
11      Elsie     Kelley
12      Sonia    Gregory
13       Leah     Curtis
14   Marjorie     Tucker
15     Rodney    Moeller
16      Velma      Lucas
17     Sidney   Burleson
18     Gladys   Hamilton
19      Frank   Waggoner
20    Marilyn       Ross
21     Jimmie  Eggleston
22         Jo     Fowler
23       Olga    Jimenez


ZADANIE 8

In [55]:
s = """SELECT country FROM country WHERE country_id IN
(SELECT country_id FROM city WHERE city_id IN
(SELECT city_id FROM address WHERE address_id IN
(SELECT address_id FROM staff)))"""


df = pd.read_sql(s, conn)
print(df)

     country
0  Australia
1     Canada


In [56]:
s = 'SELECT first_name FROM staff'
df = pd.read_sql(s, conn)
print(df)


  first_name
0       Mike
1        Jon


ZADANIE 9

In [57]:
s = """SELECT COUNT(first_name) FROM staff WHERE address_id IN
(SELECT address_id FROM address WHERE city_id IN
(SELECT city_id FROM city WHERE country_id IN
(SELECT country_id FROM country WHERE country IN (%(Argentina)s, %(Spain)s))))"""

df = pd.read_sql(s, conn, params={'Argentina': 'Argentina', 'Spain': 'Spain'})
print(df)

   count
0      0


ZADANIE 10

In [58]:
s = """SELECT name FROM category WHERE category_id IN
(SELECT category_id FROM film_category WHERE film_id IN
(SELECT film_id FROM inventory WHERE inventory_id IN
(SELECT inventory_id FROM rental WHERE customer_id IN
(SELECT customer_id FROM customer))))"""

df = pd.read_sql(s, conn)
print(df)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


ZADANIE 11

In [59]:
s = '''
SELECT name FROM category WHERE category_id IN
(SELECT category_id FROM film_category WHERE film_id IN
(SELECT film_id FROM inventory WHERE inventory_id IN
(SELECT inventory_id FROM rental WHERE customer_id IN
(SELECT customer_id FROM customer WHERE address_id IN
(SELECT address_id FROM address WHERE city_id IN
(SELECT city_id FROM city WHERE country_id IN
(SELECT country_id FROM country WHERE country=%(United States)s)))))))
'''

df = pd.read_sql(s, conn, params={'United States': 'United States'})
print(df)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


ZADANIE 12

In [60]:
s = '''
SELECT title FROM film WHERE film_id IN
(SELECT film_id FROM film_actor WHERE actor_id IN
(SELECT actor_id FROM actor WHERE
(first_name=(%(Olympia)s) AND last_name=%(Pfeiffer)s) OR
(first_name=(%(Julia)s) AND last_name=%(Zellweger)s) OR
(first_name=(%(Ellen)s) AND last_name=%(Presley)s)))
'''
#WHERE (first_name, last_name) IN (%(Uma)s, %(Wood)s)

df = pd.read_sql(s, conn, params={'Olympia': 'Olympia', 'Pfeiffer': 'Pfeiffer', 'Julia': 'Julia', 'Zellweger': 'Zellweger', 'Ellen': 'Ellen', 'Presley': 'Presley'})
print(df)

                title
0      Express Lonely
1     Bilko Anonymous
2    Effect Gladiator
3   Contact Anonymous
4         Others Soup
..                ...
64      Hanky October
65    Tourist Pelican
66   Detective Vision
67       None Spiking
68     Homeward Cider

[69 rows x 1 columns]
